# 0. Install
- 설치 후 재시작 필요

In [ ]:
!pip install -qq -U transformers accelerate
!pip install -qq datasets
!pip install -qq peft
!pip install -qq bitsandbytes
!pip install -qq trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 72.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 17.7 MB/s eta 0:00:00


# 0. Env

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import glob
import json
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split

import torch
from datasets import Dataset
from transformers import (AutoTokenizer,
                          AutoModelForCausalLM,
                          BitsAndBytesConfig,
                          pipeline,
                          TrainingArguments)
from peft import (LoraConfig,
                  PeftModel)
from trl import SFTTrainer

In [ ]:
# gradient 계산을 안하도록
torch.enable_grad(False)

<function torch.utils._contextlib.context_decorator.<locals>.decorate_context>

In [ ]:
# change work dir
work_dir = "/content/drive/MyDrive/rag-practice"

In [ ]:
# add import path
import sys
sys.path.append(work_dir)

In [ ]:
!ls {work_dir}

checkpoints  data  llm-query-decomposition.ipynb  llm-question-split.ipynb


# 1. Data

In [ ]:
fn_list = glob.glob(f"{work_dir}/data/decomposition_data/*.json")
fn_list = sorted(fn_list)
fn_list

['/content/drive/MyDrive/rag-practice/data/decomposition_data/1.json',
 '/content/drive/MyDrive/rag-practice/data/decomposition_data/100_data(c).json',
 '/content/drive/MyDrive/rag-practice/data/decomposition_data/100_data2(c).json',
 '/content/drive/MyDrive/rag-practice/data/decomposition_data/100_data3(c).json',
 '/content/drive/MyDrive/rag-practice/data/decomposition_data/100_data4(c).json',
 '/content/drive/MyDrive/rag-practice/data/decomposition_data/2.json',
 '/content/drive/MyDrive/rag-practice/data/decomposition_data/3.json',
 '/content/drive/MyDrive/rag-practice/data/decomposition_data/4.json',
 '/content/drive/MyDrive/rag-practice/data/decomposition_data/group_1_1124.json']

In [ ]:
dataset = {"question": [], "result": []}
for fn in fn_list:
    with open(fn) as f:
        data = json.load(f)
    for row in data:
        decomposed_questions = data[row]["decomposed_questions"]
        for _id in decomposed_questions:
            dataset["question"].append(decomposed_questions[_id]["question"])
            dataset["result"].append("\n".join(decomposed_questions[_id]["result"]))
df = pd.DataFrame(dataset)
df

,question,result
0,"칠리 크래브의 이름은 무엇이며, 그 뜻은 무엇인가?",1. 칠리 크래브의 이름은 무엇인가?\n2. 칠리 크래브의 뜻은 무엇인가?
1,"칠리 크래브를 처음 만들었던 사람은 누구이며, 언제부터 시작했는가?",1. 칠리 크래브를 처음 만들었던 사람은 누구인가?\n2. 칠리 크래브가 언제부터 ...
2,"칠리 크래브를 만드는 데 사용되는 주요 재료는 무엇이며, 그 중에서 어떤 재료를 먼...",1. 칠리 크래브를 만드는 데 사용되는 주요 재료는 무엇인가?\n2. 칠리 크래브를...
3,"총괄제어란 무엇이며, 철도에서 어떻게 사용되는지 설명해 주세요.",1. 총괄제어란 무엇인가?\n2. 철도에서 총괄제어의 활용 방법은 무엇인가?
4,"과거의 전동차는 어떻게 제어되었으며, 그 방식의 문제점은 무엇이었는지 알려주세요.",1. 전동차의 과거 제어 방식은 무엇이었나요?\n2. 전동차의 과거 제어 방식의 문...
...,...,...
1243,"게임보이 어드밴스 SP는 어떤 종류의 게임을 지원하며, 헤드폰을 사용하는 방법은 무...",1. 게임보이 어드밴스 SP가 지원하는 게임 종류는 무엇인가?\n2. 헤드폰을 게임...
1244,카나크는 어디에 위치하고 있으며 인구는 얼마인가?,1. 카나크 위치는 어디인가?\n2. 카나크 인구는 얼마인가?
1245,"카나크 마을은 어떻게 조성되었으며, 현재의 시설은 무엇이 있는가?",1. 카나크 마을은 어떻게 조성되었는가?\n2. 카나크 마을의 현재의 시설은 무엇이...
1246,"카나크 마을에는 어떤 도로와 교통 수단이 있으며, 의료 시설은 무엇이 있는가?",1. 카나크 마을의 도로 및 교통 수단은 무엇인가?\n2. 카나크 마을에 있는 의료...


In [ ]:
df = df[["question", "result"]]
df

,question,result
0,"칠리 크래브의 이름은 무엇이며, 그 뜻은 무엇인가?",1. 칠리 크래브의 이름은 무엇인가?\n2. 칠리 크래브의 뜻은 무엇인가?
1,"칠리 크래브를 처음 만들었던 사람은 누구이며, 언제부터 시작했는가?",1. 칠리 크래브를 처음 만들었던 사람은 누구인가?\n2. 칠리 크래브가 언제부터 ...
2,"칠리 크래브를 만드는 데 사용되는 주요 재료는 무엇이며, 그 중에서 어떤 재료를 먼...",1. 칠리 크래브를 만드는 데 사용되는 주요 재료는 무엇인가?\n2. 칠리 크래브를...
3,"총괄제어란 무엇이며, 철도에서 어떻게 사용되는지 설명해 주세요.",1. 총괄제어란 무엇인가?\n2. 철도에서 총괄제어의 활용 방법은 무엇인가?
4,"과거의 전동차는 어떻게 제어되었으며, 그 방식의 문제점은 무엇이었는지 알려주세요.",1. 전동차의 과거 제어 방식은 무엇이었나요?\n2. 전동차의 과거 제어 방식의 문...
...,...,...
1243,"게임보이 어드밴스 SP는 어떤 종류의 게임을 지원하며, 헤드폰을 사용하는 방법은 무...",1. 게임보이 어드밴스 SP가 지원하는 게임 종류는 무엇인가?\n2. 헤드폰을 게임...
1244,카나크는 어디에 위치하고 있으며 인구는 얼마인가?,1. 카나크 위치는 어디인가?\n2. 카나크 인구는 얼마인가?
1245,"카나크 마을은 어떻게 조성되었으며, 현재의 시설은 무엇이 있는가?",1. 카나크 마을은 어떻게 조성되었는가?\n2. 카나크 마을의 현재의 시설은 무엇이...
1246,"카나크 마을에는 어떤 도로와 교통 수단이 있으며, 의료 시설은 무엇이 있는가?",1. 카나크 마을의 도로 및 교통 수단은 무엇인가?\n2. 카나크 마을에 있는 의료...


In [ ]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=1234)

In [ ]:
df_train.to_csv(f"{work_dir}/data/decomposition_train.csv")

In [ ]:
df_test.to_csv(f"{work_dir}/data/decomposition_test.csv")

# 2. Gemma-2-2b

In [ ]:
# 추론에 사용할 LLM
MODEL_ID = 'google/gemma-2-2b-it'
# hugging face access token
HF_TOKEN = ""

## 2.1. Dataset

In [ ]:
df_train = pd.read_csv(f"{work_dir}/data/decomposition_train.csv")

In [ ]:
train_dataset = Dataset.from_pandas(df_train)

In [ ]:
def gemma_train_prompt(example):
    prompt_list = []
    for i in range(len(example['question'])):
        question = example['question'][i]
        result = example['result'][i]
        prompt = """<bos><start_of_turn>user
당신은 하나의 복잡한 질문을 2개 이상의 단순한 정보를 물어보는 질문으로 변경해 주는 역할을 하는 어시스턴트입니다.
다음 질문을 2개 이상의 단순한 정보를 물어보는 질문으로 변경해주세요.:

질문: {}<end_of_turn>
<start_of_turn>model
{}<end_of_turn><eos>""".format(question, result)
        prompt_list.append(prompt)
    return prompt_list

In [ ]:
for example in train_dataset:
    example['question'] = [example['question']]
    example['result'] = [example['result']]
    text = gemma_train_prompt(example)
    print(text[0])
    break

<bos><start_of_turn>user
당신은 하나의 복잡한 질문을 2개 이상의 단순한 정보를 물어보는 질문으로 변경해 주는 역할을 하는 어시스턴트입니다.
다음 질문을 2개 이상의 단순한 정보를 물어보는 질문으로 변경해주세요.:

질문: 맨홀의 크기와 재질은 어떻게 되나요?<end_of_turn>
<start_of_turn>model
1. 맨홀의 크기는 어떻게 되나요?
2. 맨홀의 재질은 어떻게 되나요?<end_of_turn><eos>


In [ ]:
df_test = pd.read_csv(f"{work_dir}/data/decomposition_test.csv")

In [ ]:
test_dataset = Dataset.from_pandas(df_test)

In [ ]:
def gemma_test_prompt(example):
    prompt_list = []
    for i in range(len(example['question'])):
        question = example['question'][i]
        prompt = """<bos><start_of_turn>user
당신은 하나의 복잡한 질문을 2개 이상의 단순한 정보를 물어보는 질문으로 변경해 주는 역할을 하는 어시스턴트입니다.
다음 질문을 2개 이상의 단순한 정보를 물어보는 질문으로 변경해주세요.:

질문: {}<end_of_turn>
<start_of_turn>model
""".format(question)
        prompt_list.append(prompt)
    return prompt_list

In [ ]:
for example in test_dataset:
    example['question'] = [example['question']]
    text = gemma_test_prompt(example)
    print(text[0])
    break

<bos><start_of_turn>user
당신은 하나의 복잡한 질문을 2개 이상의 단순한 정보를 물어보는 질문으로 변경해 주는 역할을 하는 어시스턴트입니다.
다음 질문을 2개 이상의 단순한 정보를 물어보는 질문으로 변경해주세요.:

질문: 브레멘의 중요성과 특징은 무엇인가? (예: 항구 도시, 축구팀 연고지 등)<end_of_turn>
<start_of_turn>model



## 2.2. Train

In [ ]:
# declare 4 bits quantize
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

In [ ]:
# load 4 bits model
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map='auto',
    quantization_config=bnb_config,
    attn_implementation='eager',
    token=HF_TOKEN
)

config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [ ]:
# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID,
                                          token=HF_TOKEN)
tokenizer.padding_side = 'right'

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [ ]:
args = TrainingArguments(
    output_dir=f"{work_dir}/checkpoints/gemma-2-2b",
    num_train_epochs = 10,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    warmup_steps=25,
    learning_rate=1e-4,
    bf16=False,
    fp16=False,
    eval_strategy="no",
    logging_strategy="steps",
    logging_steps=23,
    save_strategy="steps",
    save_steps=23,
    save_total_limit=3,
    report_to="none",
    push_to_hub=False
)

In [ ]:
# lora config
lora_config = LoraConfig(
    r=6,
    lora_alpha=8,
    lora_dropout=0.05,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    # max_seq_length=512,
    args=args,
    peft_config=lora_config,
    formatting_func=gemma_train_prompt,
)

<ipython-input-28-c858d52100d2>:1: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Map:   0%|          | 0/998 [00:00<?, ? examples/s]

In [ ]:
# train
os.environ["WANDB_DISABLED"] = "true"
model.config.use_cache = False
trainer.train()

Step,Training Loss
23,7.945600
46,3.188000
69,2.535200
92,2.214800
115,2.084300
138,1.829900
161,1.763400
184,1.731200
207,1.681300
230,1.652700


/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:716: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-6781f5f4-227ad51d6820888a0444c725;8ef2271a-4355-4cd0-b132-0cc1f553fc3d)

Cannot access gated repo for url https://huggingface.co/google/gemma-2-2b-it/resolve/main/config.json.
Access to model google/gemma-2-2b-it is restricted. You must have access to it and be authenticated to access it. Please log in. - silently ignoring the lookup for the file config.json in google/gemma-2-2b-it.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:246: UserWarning: Could not find a config file in google/gemma-2-2b-it - will assume that the vocabulary was not modified.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:716: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-6781f63a-0f6102d102b833dd5bd641ca;f51edf5c-3adf

TrainOutput(global_step=1240, training_loss=1.055654377322043, metrics={'train_runtime': 3699.8666, 'train_samples_per_second': 2.697, 'train_steps_per_second': 0.335, 'total_flos': 2.0841186279158784e+16, 'train_loss': 1.055654377322043, 'epoch': 9.993987975951903})

In [ ]:
trainer.model.save_pretrained(f"{work_dir}/checkpoints/gemma-2-2b/checkpoint-delta")

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:716: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-67820423-423b5771061120bd1ce2b0ac;41540d17-3c4f-42fc-8156-c1acacf9fd0d)

Cannot access gated repo for url https://huggingface.co/google/gemma-2-2b-it/resolve/main/config.json.
Access to model google/gemma-2-2b-it is restricted. You must have access to it and be authenticated to access it. Please log in. - silently ignoring the lookup for the file config.json in google/gemma-2-2b-it.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:246: UserWarning: Could not find a config file in google/gemma-2-2b-it - will assume that the vocabulary was not modified.
  warnings.warn(


In [ ]:
# original model load (before finetuned)
model = AutoModelForCausalLM.from_pretrained(MODEL_ID,
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             token=HF_TOKEN)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# merge : original + delta wieght
model = PeftModel.from_pretrained(model,
                                  f"{work_dir}/checkpoints/gemma-2-2b/checkpoint-delta",
                                  device_map='auto',
                                  torch_dtype=torch.float16,
                                  token=HF_TOKEN)
model = model.merge_and_unload()

In [ ]:
# save fine-tunned model
model.save_pretrained(f"{work_dir}/checkpoints/gemma-2-2b/checkpoint-merged")

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2810: UserWarning: Attempting to save a model with offloaded modules. Ensure that unallocated cpu memory exceeds the `shard_size` (5GB default)
  warnings.warn(


Saving checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
!ls {work_dir}/checkpoints/gemma-2-2b

checkpoint-1196  checkpoint-1219  checkpoint-1240  checkpoint-delta  checkpoint-merged


In [ ]:
!ls {work_dir}/checkpoints/gemma-2-2b/checkpoint-merged

config.json		model-00001-of-00002.safetensors  model.safetensors.index.json
generation_config.json	model-00002-of-00002.safetensors


## 2.3. Test with SFT

In [ ]:
model_fn = f"{work_dir}/checkpoints/gemma-2-2b/checkpoint-merged"

In [ ]:
# declare 4 bits quantize
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

In [ ]:
# load 4 bits model
model = AutoModelForCausalLM.from_pretrained(
    model_fn,
    device_map='auto',
    quantization_config=bnb_config,
    attn_implementation='eager',
    token=HF_TOKEN
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID,
                                          token=HF_TOKEN)
tokenizer.padding_side = 'right'

In [ ]:
# pipeline
pipe = pipeline("text-generation",
                model=model,
                tokenizer=tokenizer,
                max_new_tokens=512)

Device set to use cuda:0


In [ ]:
def gen_response(pipe, question):
    prompt = gemma_test_prompt({'question': [question]})[0]
    # print(prompt)

    outputs = pipe(
        prompt,
        do_sample=True,
        temperature=0.2,
        top_k=50,
        top_p=0.95,
        add_special_tokens=True
    )
    return outputs[0]["generated_text"][len(prompt):]

In [ ]:
responses = []
for row in tqdm(test_dataset):
    question = row['question']
    response = gen_response(pipe, question)
    print(question.strip())
    print(response.strip())
    print("*" * 100)
    responses.append(response.strip())

  0%|          | 0/250 [00:00<?, ?it/s]

The 'batch_size' attribute of HybridCache is deprecated and will be removed in v4.49. Use the more precisely named 'self.max_batch_size' attribute instead.


브레멘의 중요성과 특징은 무엇인가? (예: 항구 도시, 축구팀 연고지 등)
1. 브레멘의 중요성은 무엇인가?
2. 브레멘의 특징은 무엇인가?
****************************************************************************************************
브라운슈바이크 공국의 면적과 인구는 얼마였나?
1. 브라운슈바이크 공국의 면적은 얼마인가?
2. 브라운슈바이크 공국의 인구는 얼마인가?
****************************************************************************************************
런 렝스 부호화(RLE)의 정의와 특징은 무엇인가?
1. 런 렝스 부호화(RLE)의 정의는 무엇인가?
2. 런 렝스 부호화(RLE)의 특징은 무엇인가?
****************************************************************************************************
디지털미디어시티역의 구조와 출구는 어떤지 설명해 주세요.
1. 디지털미디어시티역 구조는 어떤 방식으로 구성되어 있나요?
2. 디지털미디어시티역 출구는 어떻게 접근할 수 있나요?
****************************************************************************************************
솔라리스 운영체제는 어떤 아키텍처를 지원하고 있으며, 그 중에서 가장 최근에 추가된 아키텍처는 무엇인가?
1. 솔라리스 운영체제가 지원하는 아키텍처 종류는 무엇인가?
2. 최근에 솔라리스 운영체제로 사용할 수 있는 아키텍처 중 가장 최신 것은 무엇인가?
****************************************************************************************************
게임보이 어드밴스 SP의 

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


제우스는 인간들이 문명에 이르는 데 없어서는 안 될 것을 숨겨 버렸는데, 그것은 무엇이며, 프로메테우스는 어떻게 불을 가져다주었는지 알려주세요.
1. 제우스가 문명에 이르는 데 필요로 하는 것은 무엇인가?
2. 프로메테우스는 어떻게 불을 가져다주었나요?
****************************************************************************************************
아시노의 산업과 주요 공장에 대해 알려주세요.
1. 아시노의 산업은 무엇이며 어떤 자원을 중심으로 발전하고 있는가?
2. 아시노의 주요 공장은 어디에 위치하고 있으며 어떤 제품을 생산하고 있는가?
****************************************************************************************************
악어의 알은 어떻게 생식하고, 부화하는 과정에서 무슨 일이 일어나는가?
1. 악어의 알 생식기관은 어떻게 작동하는가?
2. 악어 알에서 부화하는 과정에서 어떤 변화가 일어나는가?
****************************************************************************************************
인종본질주의자들은 어떤 저작을 인용하며, 그 중 일부는 어떻게 받아들여지는가?
1. 인종본질주의자들은 어떤 저작을 인용하는가?
2. 인종본질주의자들의 저작에 어떤 분류가 있나?
****************************************************************************************************
AC 밀란의 홈 구장은 어디이며, 수용 인원은 얼마인가?
1. AC 밀란의 홈 구장은 어디인가?
2. AC 밀란의 홈 구장의 수용 인원은 얼마인가?
**************************************************************

In [ ]:
df_test["gemma-2-2b with SFT"] = responses
df_test

,Unnamed: 0,question,result,gemma-2-2b with SFT
0,400,"브레멘의 중요성과 특징은 무엇인가? (예: 항구 도시, 축구팀 연고지 등)",1. 브레멘의 중요성은 무엇인가?\n2. 브레멘의 특징은 무엇인가?,1. 브레멘의 중요성은 무엇인가?\n2. 브레멘의 특징은 무엇인가?
1,272,브라운슈바이크 공국의 면적과 인구는 얼마였나?,1. 브라운슈바이크 공국의 면적은 얼마인가?\n2. 브라운슈바이크 공국의 인구는 얼...,1. 브라운슈바이크 공국의 면적은 얼마인가?\n2. 브라운슈바이크 공국의 인구는 얼...
2,495,런 렝스 부호화(RLE)의 정의와 특징은 무엇인가?,1. 런 렝스 부호화(RLE)의 정의는 무엇인가?\n2. 런 렝스 부호화(RLE)의...,1. 런 렝스 부호화(RLE)의 정의는 무엇인가?\n2. 런 렝스 부호화(RLE)의...
3,330,디지털미디어시티역의 구조와 출구는 어떤지 설명해 주세요.,1. 디지털미디어시티역의 구조는 어떻게 되어 있나요?\n2. 디지털미디어시티역의 출...,1. 디지털미디어시티역 구조는 어떤 방식으로 구성되어 있나요?\n2. 디지털미디어시...
4,228,"솔라리스 운영체제는 어떤 아키텍처를 지원하고 있으며, 그 중에서 가장 최근에 추가된...",1. 솔라리스 운영체제는 어떤 아키텍처를 지원하고 있나요?\n2. 솔라리스 운영체제...,1. 솔라리스 운영체제가 지원하는 아키텍처 종류는 무엇인가?\n2. 최근에 솔라리스...
...,...,...,...,...
245,351,"고령군의 지리적 특징은 무엇이며, 낙동강과 관련된 지류들은 어떤지?",1. 고령군의 지리적 특징은 무엇인가\n2. 고령군의 지리 중 낙동강과 관련된 지류...,1. 고령군의 지리적 특징은 무엇인가요?\n2. 고령군 면적에 포함된 낙동강과 관련...
246,305,백일홍의 학명과 영명은 무엇인가?,1. 백일홍의 학명은 무엇인가?\n2. 백일홍의 영명은 무엇인가?,1. 백일홍의 학명은 무엇인가?\n2. 백일홍의 영명은 무엇인가?
247,636,"조모는 사마씨의 권력을 어떻게 비판했으며, 그 결과는 무엇이었나요?",1. 사마씨의 권력 비판 방법은 무엇이었나요?\n2. 조모의 권력 비판으로 인한 결...,1. 조모가 사마씨의 권력을 어떻게 비판했는가?\n2. 조모가 사마씨의 권력을 비판...
248,1135,"전역은 무엇을 의미하고, 전구와는 어떤 관계를 가지고 있는가?",1. 전역은 무엇을 의미하고 있는가?\n2. 전구와 전역의 관계는 무엇인가?,1. 전역이란 무엇을 의미하는가?\n2. 전구와의 관계는 무엇인가?


## 2.4. Test without SFT

In [ ]:
# declare 4 bits quantize
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

In [ ]:
# load 4 bits model
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map='auto',
    quantization_config=bnb_config,
    attn_implementation='eager',
    token=HF_TOKEN
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID,
                                          token=HF_TOKEN)
tokenizer.padding_side = 'right'

In [ ]:
# pipeline
pipe = pipeline("text-generation",
                model=model,
                tokenizer=tokenizer,
                max_new_tokens=512)

Device set to use cuda:0


In [ ]:
def gen_response(pipe, question):
    prompt = gemma_test_prompt({'question': [question]})[0]
    # print(prompt)

    outputs = pipe(
        prompt,
        do_sample=True,
        temperature=0.2,
        top_k=50,
        top_p=0.95,
        add_special_tokens=True
    )
    return outputs[0]["generated_text"][len(prompt):]

In [ ]:
responses = []
for row in tqdm(test_dataset):
    question = row['question']
    response = gen_response(pipe, question)
    print(question.strip())
    print(response.strip())
    print("*" * 100)
    responses.append(response.strip())

  0%|          | 0/250 [00:00<?, ?it/s]

브레멘의 중요성과 특징은 무엇인가? (예: 항구 도시, 축구팀 연고지 등)
브레멘의 중요성과 특징에 대해 알려주시겠어요? 

1. **브레멘의 중요성은 무엇인가요?** 
2. **브레멘의 특징은 무엇인가요?** 


이 질문들은 브레멘의 중요성과 특징에 대한 답변을 얻을 수 있도록 돕습니다.
****************************************************************************************************
브라운슈바이크 공국의 면적과 인구는 얼마였나?
브라운슈바이크 공국의 면적과 인구에 대해 알려주시겠어요? 

1. **브라운슈바이크 공국의 면적은 얼마인가요?**
2. **브라운슈바이크 공국의 인구는 얼마인가요?**
****************************************************************************************************
런 렝스 부호화(RLE)의 정의와 특징은 무엇인가?
런 렝스 부호화(RLE)에 대해 알고 싶어요. 

1. **RLE를 어떻게 정의할 수 있나요?** 
2. **RLE의 특징은 무엇인가요?** 


이렇게 질문을 변경하면, 좀 더 구체적인 정보를 얻을 수 있을 것입니다.
****************************************************************************************************
디지털미디어시티역의 구조와 출구는 어떤지 설명해 주세요.
디지털미디어시티역의 구조와 출구에 대해 설명해 주시겠어요? 

1. **디지털미디어시티역의 구조는 어떤 방식으로 구성되어 있나요?** (예: 탑승 가능한 측면, 층수, 탑승/하차 측면 등)
2. **디지털미디어시티역의 출구는 어디에 있는지, 어떤 방향으로 나가는지 설명해주세요.** (예: 외부 입구, 지하철역 출구, 탑승/하차 측면 등)
***************************************

In [ ]:
df_test["gemma-2-2b w/o SFT"] = responses
df_test

,Unnamed: 0,question,result,gemma-2-2b with SFT,gemma-2-2b w/o SFT
0,400,"브레멘의 중요성과 특징은 무엇인가? (예: 항구 도시, 축구팀 연고지 등)",1. 브레멘의 중요성은 무엇인가?\n2. 브레멘의 특징은 무엇인가?,1. 브레멘의 중요성은 무엇인가?\n2. 브레멘의 특징은 무엇인가?,브레멘의 중요성과 특징에 대해 알려주시겠어요? \n\n1. **브레멘의 중요성은 무...
1,272,브라운슈바이크 공국의 면적과 인구는 얼마였나?,1. 브라운슈바이크 공국의 면적은 얼마인가?\n2. 브라운슈바이크 공국의 인구는 얼...,1. 브라운슈바이크 공국의 면적은 얼마인가?\n2. 브라운슈바이크 공국의 인구는 얼...,브라운슈바이크 공국의 면적과 인구에 대해 알려주시겠어요? \n\n1. **브라운슈바...
2,495,런 렝스 부호화(RLE)의 정의와 특징은 무엇인가?,1. 런 렝스 부호화(RLE)의 정의는 무엇인가?\n2. 런 렝스 부호화(RLE)의...,1. 런 렝스 부호화(RLE)의 정의는 무엇인가?\n2. 런 렝스 부호화(RLE)의...,런 렝스 부호화(RLE)에 대해 알고 싶어요. \n\n1. **RLE를 어떻게 정의...
3,330,디지털미디어시티역의 구조와 출구는 어떤지 설명해 주세요.,1. 디지털미디어시티역의 구조는 어떻게 되어 있나요?\n2. 디지털미디어시티역의 출...,1. 디지털미디어시티역 구조는 어떤 방식으로 구성되어 있나요?\n2. 디지털미디어시...,디지털미디어시티역의 구조와 출구에 대해 설명해 주시겠어요? \n\n1. **디지털미...
4,228,"솔라리스 운영체제는 어떤 아키텍처를 지원하고 있으며, 그 중에서 가장 최근에 추가된...",1. 솔라리스 운영체제는 어떤 아키텍처를 지원하고 있나요?\n2. 솔라리스 운영체제...,1. 솔라리스 운영체제가 지원하는 아키텍처 종류는 무엇인가?\n2. 최근에 솔라리스...,솔라리스 운영체제가 어떤 아키텍처를 지원하는지 알려주세요. \n그리고 솔라리스 운영...
...,...,...,...,...,...
245,351,"고령군의 지리적 특징은 무엇이며, 낙동강과 관련된 지류들은 어떤지?",1. 고령군의 지리적 특징은 무엇인가\n2. 고령군의 지리 중 낙동강과 관련된 지류...,1. 고령군의 지리적 특징은 무엇인가요?\n2. 고령군 면적에 포함된 낙동강과 관련...,"고령군의 지리적 특징은 무엇인가요? \n\n고령군의 지리적 특징에 대해 알려주시면,..."
246,305,백일홍의 학명과 영명은 무엇인가?,1. 백일홍의 학명은 무엇인가?\n2. 백일홍의 영명은 무엇인가?,1. 백일홍의 학명은 무엇인가?\n2. 백일홍의 영명은 무엇인가?,백일홍의 학명과 영명은 무엇인가요? \n\n1. **백일홍의 학명은 무엇인가요?**...
247,636,"조모는 사마씨의 권력을 어떻게 비판했으며, 그 결과는 무엇이었나요?",1. 사마씨의 권력 비판 방법은 무엇이었나요?\n2. 조모의 권력 비판으로 인한 결...,1. 조모가 사마씨의 권력을 어떻게 비판했는가?\n2. 조모가 사마씨의 권력을 비판...,조모와 사마씨의 관계에 대해 알려주시면 됩니다. 조모는 사마씨의 권력에 대해 어떻게...
248,1135,"전역은 무엇을 의미하고, 전구와는 어떤 관계를 가지고 있는가?",1. 전역은 무엇을 의미하고 있는가?\n2. 전구와 전역의 관계는 무엇인가?,1. 전역이란 무엇을 의미하는가?\n2. 전구와의 관계는 무엇인가?,"전역이 무엇을 의미하는지, 그리고 전구와의 관계는 어떤가요? \n\n1. **전역이..."


In [ ]:
df_test.to_excel(f"{work_dir}/data/gemma_2_2b_results_out.xlsx")